**Querying Source** 


In [0]:
%sql 
select *from pyspark_cata.source.products 

id,name,price,catagory,updatedDate
1,iPhone,1000,electronics,2025-11-23T00:30:11.517Z
2,Macbook,2000,electronics,2025-11-23T00:30:11.517Z
3,Airpods,500,electronics,2025-11-23T00:30:11.517Z
4,Shirt,100,clothing,2025-11-23T00:30:11.517Z
5,Pants,150,clothing,2025-11-23T00:30:11.517Z


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
#create delta object on top of the sink location
df = spark.sql("select * from pyspark_cata.source.products")

#Dedup
df = df.withColumn("dedup", row_number().over(Window.partitionBy("id").orderBy(desc('updatedDate'))))
df= df.filter(col('dedup')==1).drop('dedup')
display(df)

id,name,price,catagory,updatedDate
1,iPhone,1000,electronics,2025-11-23T00:30:11.517Z
2,Macbook,2000,electronics,2025-11-23T00:30:11.517Z
3,Airpods,500,electronics,2025-11-23T00:30:11.517Z
4,Shirt,100,clothing,2025-11-23T00:30:11.517Z
5,Trousers,150,clothing,2025-11-24T21:09:42.783Z


## **UPSERT**

In [0]:
#Creating Delta Opject 
from delta.tables import DeltaTable
#if spark.catalog.tableExists() or #try:
if len(dbutils.fs.ls("/Volumes/pyspark_cata/source/db_volume/products_sink/"))>0:

   
    dlt_obj = DeltaTable.forPath(spark,"/Volumes/pyspark_cata/source/db_volume/products_sink/")

    # Upsert
    dlt_obj.alias("trg").merge(
        df.alias("src"),
        "src.id = trg.id"
    ) \
    .whenMatchedUpdateAll(condition="src.updatedDate >= trg.updatedDate")\
    .whenNotMatchedInsertAll()\
    .execute()
    print ("This is upserting now")  
#except:
else:
    df.write.format("delta")\
        .mode("Overwrite")\
        .save("/Volumes/pyspark_cata/source/db_volume/products_sink/")


This is upserting now


In [0]:
%sql
SELECT *FROM delta.`/Volumes/pyspark_cata/source/db_volume/products_sink/`

id,name,price,catagory,updatedDate
1,iPhone,1000,electronics,2025-11-23T00:30:11.517Z
2,Macbook,2000,electronics,2025-11-23T00:30:11.517Z
3,Airpods,500,electronics,2025-11-23T00:30:11.517Z
4,Shirt,100,clothing,2025-11-23T00:30:11.517Z
5,Trousers,150,clothing,2025-11-24T21:09:42.783Z
5,Trousers,150,clothing,2025-11-24T21:09:42.783Z
